### Co zrobione
  ### 'Data_pierwszej_rejestracji'
    - Naprawienie formatowania 'Data_pierwszej_rejestracji'
    - Rozdzielenie 'Data_pierwszej_rejestracji' na kolumny 'Miesiac_pierwszej_rejestracji' i 'Rok_pierwszej_rejestracji'
    - Zmiana nieprawindłowych lat z 'Rok_pierwszej_rejestracji' poprzez iterowanie po indeksach roku i zamianie ich na wartości o odpowiadającym indeksie z 'Rok_produkcji'
    - Usunięcie atrybutu 'Data_pierwszej_rejestracji' (Warto się zastanowić czy zostawić usunięte, czy może zrobić merge 'Miesiac_pierwszej_rejestracji' i 'Rok_pierwszej_rejestracji')
  ### 'Generacja_pojazdu'
    - Chyba nie trzeba czyścić, braki końcowego roku zdają się być schematyczne a nie wybiórcze.  Możliwe jednak że coś przegapiłem

  ### 'Cena' i 'Waluta'
    - Wartości ceny wyrażonej w EUR zostały przekonwertowane na PLN przy pomocy API NBP wedle średniego kursu walut dnia wystawienia pojazdu lub najbliższej dostępnej dacie wśród pozostałych wartości z EUR
    - Atrybut 'Waluta' usunięty skoro  wszystkie wartości wyrażone są w PLN

  ### 'Wyposazenie'
    - Wykonany One Hot Encoding wyposażenia
    - Usunięty atrybut 'Wyposazenie'

In [2072]:
from enum import unique

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.f2py.auxfuncs import throw_error
from sklearn.preprocessing import TargetEncoder

In [2073]:
df_car_sales_train_data = pd.read_csv('Data/sales_ads_train.csv',
    dtype={
        'Cena' : 'float',
        'Waluta': 'category',
        'Stan': 'category',
        'Marka_pojazdu': 'string',
        'Model_pojazdu': 'string',
        'Wersja_pojazdu': 'string',
        'Generacja_pojazdu': 'string',
        'Rok_produkcji': 'float64',
        'Przebieg_km': 'float',
        'Moc_KM': 'float',
        'Pojemnosc_cm3': 'float',
        'Rodzaj_paliwa': 'category',
        'Emisja_CO2': 'float',
        'Naped': 'category',
        'Skrzynia_biegow': 'category',
        'Typ_nadwozia': 'category',
        'Liczba_drzwi': 'float64',
        'Kolor': 'category',
        'Kraj_pochodzenia': 'category',
        'Pierwszy': 'category',
        'Lokalizacja_oferty': 'string',
        'Wyposazenie': 'string'

},
                           index_col="ID", parse_dates=['Data_pierwszej_rejestracji','Data_publikacji_oferty'], dayfirst=True)
df_car_sales_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  float64       
 1   Waluta                      132021 non-null  category      
 2   Stan                        132075 non-null  category      
 3   Marka_pojazdu               132046 non-null  string        
 4   Model_pojazdu               132088 non-null  string        
 5   Wersja_pojazdu              87336 non-null   string        
 6   Generacja_pojazdu           93737 non-null   string        
 7   Rok_produkcji               131990 non-null  float64       
 8   Przebieg_km                 131394 non-null  float64       
 9   Moc_KM                      131664 non-null  float64       
 10  Pojemnosc_cm3               130711 non-null  float64       
 11  Rodzaj_paliwa               131987 non-null 

In [2074]:
df_car_sales_test_data = pd.read_csv('Data/sales_ads_test.csv',
    dtype={
        'Cena' : 'float',
        'Waluta': 'category',
        'Stan': 'category',
        'Marka_pojazdu': 'string',
        'Model_pojazdu': 'string',
        'Wersja_pojazdu': 'string',
        'Generacja_pojazdu': 'string',
        'Rok_produkcji': 'float64',
        'Przebieg_km': 'float',
        'Moc_KM': 'float',
        'Pojemnosc_cm3': 'float',
        'Rodzaj_paliwa': 'category',
        'Emisja_CO2': 'float',
        'Naped': 'category',
        'Skrzynia_biegow': 'category',
        'Typ_nadwozia': 'category',
        'Liczba_drzwi': 'float64',
        'Kolor': 'category',
        'Kraj_pochodzenia': 'category',
        'Pierwszy': 'category',
        'Lokalizacja_oferty': 'string',
        'Wyposazenie': 'string'
},
                           index_col="ID", parse_dates=['Data_pierwszej_rejestracji','Data_publikacji_oferty'], dayfirst=True)
df_car_sales_test_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 72907 entries, 1 to 72907
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Waluta                      72907 non-null  category      
 1   Stan                        72907 non-null  category      
 2   Marka_pojazdu               72907 non-null  string        
 3   Model_pojazdu               72907 non-null  string        
 4   Wersja_pojazdu              48394 non-null  string        
 5   Generacja_pojazdu           51734 non-null  string        
 6   Rok_produkcji               72907 non-null  float64       
 7   Przebieg_km                 72577 non-null  float64       
 8   Moc_KM                      72678 non-null  float64       
 9   Pojemnosc_cm3               72221 non-null  float64       
 10  Rodzaj_paliwa               72907 non-null  category      
 11  Emisja_CO2                  32946 non-null  float64       


# UWAGA!!
Łączenie danych testowych i treningowych tylko na czas czyszczenia stringów i formatowań!!!

In [2075]:
df_car_sales_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  float64       
 1   Waluta                      132021 non-null  category      
 2   Stan                        132075 non-null  category      
 3   Marka_pojazdu               132046 non-null  string        
 4   Model_pojazdu               132088 non-null  string        
 5   Wersja_pojazdu              87336 non-null   string        
 6   Generacja_pojazdu           93737 non-null   string        
 7   Rok_produkcji               131990 non-null  float64       
 8   Przebieg_km                 131394 non-null  float64       
 9   Moc_KM                      131664 non-null  float64       
 10  Pojemnosc_cm3               130711 non-null  float64       
 11  Rodzaj_paliwa               131987 non-null 

In [2076]:
df_car_sales_train_data["Wersja_pojazdu"].value_counts()

Wersja_pojazdu
2.0 TDI                                    596
1.6                                        540
Standard                                   418
1.2                                        392
2.0 TDCi Titanium                          390
                                          ... 
BlueHDi FAP 150 Stop&Start                   1
330i GPF xDrive Sport Line Shadow sport      1
2.4 Executive Navi                           1
1.6 Multijet Beats Edition S&S               1
1.9 TDI Sport Limited                        1
Name: count, Length: 16014, dtype: Int64

# Ustaw dane które przetwarzasz

In [2077]:
df_car_sales_test_data['Cena'] = pd.Series()
# df_car_sales = pd.concat([df_car_sales_train_data, df_car_sales_test_data], ignore_index = True)
df_car_sales = df_car_sales_train_data
# df_car_sales = df_car_sales_test_data
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  float64       
 1   Waluta                      132021 non-null  category      
 2   Stan                        132075 non-null  category      
 3   Marka_pojazdu               132046 non-null  string        
 4   Model_pojazdu               132088 non-null  string        
 5   Wersja_pojazdu              87336 non-null   string        
 6   Generacja_pojazdu           93737 non-null   string        
 7   Rok_produkcji               131990 non-null  float64       
 8   Przebieg_km                 131394 non-null  float64       
 9   Moc_KM                      131664 non-null  float64       
 10  Pojemnosc_cm3               130711 non-null  float64       
 11  Rodzaj_paliwa               131987 non-null 

In [2078]:
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  float64       
 1   Waluta                      132021 non-null  category      
 2   Stan                        132075 non-null  category      
 3   Marka_pojazdu               132046 non-null  string        
 4   Model_pojazdu               132088 non-null  string        
 5   Wersja_pojazdu              87336 non-null   string        
 6   Generacja_pojazdu           93737 non-null   string        
 7   Rok_produkcji               131990 non-null  float64       
 8   Przebieg_km                 131394 non-null  float64       
 9   Moc_KM                      131664 non-null  float64       
 10  Pojemnosc_cm3               130711 non-null  float64       
 11  Rodzaj_paliwa               131987 non-null 

# Usuwanie zbędnych atrybutów

In [2079]:
df_car_sales.drop(columns=["Emisja_CO2", "Liczba_drzwi"], inplace=True)

## Pierwszy właściciel

In [2080]:
df_car_sales_test_data["Pierwszy_wlasciciel"].replace({"Yes": 1, "No": 0}, inplace=True)
df_car_sales_train_data["Pierwszy_wlasciciel"].replace({"Yes": 1, "No": 0}, inplace=True)

C:\Users\vipcy\AppData\Local\Temp\ipykernel_18164\4229917566.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_car_sales_test_data["Pierwszy_wlasciciel"].replace({"Yes": 1, "No": 0}, inplace=True)
C:\Users\vipcy\AppData\Local\Temp\ipykernel_18164\4229917566.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcast

### Czyszczenie i konwersja 'Data_pierwszej_rejestracji' do typu 'datetetime64'

In [2081]:
df_car_sales['Data_pierwszej_rejestracji']

ID
1                NaN
2         16/06/2010
3                NaN
4         27/11/2007
5         20/05/2013
             ...    
135393    22/08/2018
135394           NaN
135395           NaN
135396           NaN
135397    31/01/2019
Name: Data_pierwszej_rejestracji, Length: 135397, dtype: object

Zakładając brak pomyłek:

 - najstarsze auto zostało wyprodukowane w 1923, lecz dane testowe zawierają w sobie nawet takie z 1915
 - dane pochodzą z 2021

In [2082]:
df_car_sales['Rok_produkcji'].describe()

count    131990.000000
mean       2012.089901
std           7.003557
min        1923.000000
25%        2008.000000
50%        2013.000000
75%        2017.000000
max        2021.000000
Name: Rok_produkcji, dtype: float64

'Data_pierwszej_rejestracji' zawiera w sobie zarówno błędne formatowanie jak i rok rejestracji

In [2083]:
df_car_sales['Data_pierwszej_rejestracji'] = df_car_sales['Data_pierwszej_rejestracji'].str.replace('//', '/')

In [2084]:
# Dzielenie daty rejestracji na kolumny dnia, miesiąca i roku
first_registration_expanded_data = df_car_sales['Data_pierwszej_rejestracji'].str.split('/', expand=True).rename(columns={0: 'Dzien_pierwszej_rejestracji', 1: 'Miesiac_pierwszej_rejestracji', 2: 'Rok_pierwszej_rejestracji'})

In [2085]:
# Łączenie datych z kolumną roku produkci pojazdu dla dalszej analizy
df_car_sales_with_expanded_registration_dates = pd.concat([df_car_sales['Rok_produkcji'], first_registration_expanded_data], axis=1)
df_car_sales_with_expanded_registration_dates

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,
1,2005.0,NaN,NaN,NaN
2,2010.0,16,06,2010
3,2015.0,NaN,NaN,NaN
4,2007.0,27,11,2007
5,2013.0,20,05,2013
...,...,...,...,...
135393,2018.0,22,08,2018
135394,2021.0,NaN,NaN,NaN
135395,NaN,NaN,NaN,NaN


In [2086]:
df_car_sales_with_expanded_registration_dates.loc[
    df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < 1940
][['Rok_produkcji', 'Rok_pierwszej_rejestracji']]

# Zastąpienie pierwszych 2 cyfr roku cyframi z roku produkcji zdaje się że rozwiąże większość konfliktów

,Rok_produkcji,Rok_pierwszej_rejestracji
ID,,
1727,2006.0,0012
1890,2016.0,0017
2265,1998.0,0098
2744,1998.0,0098
3999,2000.0,0200
...,...,...
130368,2009.0,0209
131035,2006.0,0021
132905,2006.0,0019


In [2087]:
# Dodać do listy zakupów
df_car_sales.iloc[32583]

Cena                                              1300000.0
Waluta                                                  PLN
Stan                                                   Used
Marka_pojazdu                                   Rolls-Royce
Model_pojazdu                                       Phantom
Wersja_pojazdu                                         <NA>
Generacja_pojazdu                                      <NA>
Rok_produkcji                                        1936.0
Przebieg_km                                         40355.0
Moc_KM                                                  NaN
Pojemnosc_cm3                                        3499.0
Rodzaj_paliwa                                      Gasoline
Naped                                           Rear wheels
Skrzynia_biegow                                      Manual
Typ_nadwozia                                      city_cars
Kolor                                                 other
Kraj_pochodzenia                        

In [2088]:
# Usuwanie ujemnych wartości z roku
df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'] = (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].
                                                                              str.replace('-', ''))

In [2089]:
# Wyświetlanie danych gdzie rok pierwszej rejestracji jest wcześniejszy niż rok produkcji
df_car_sales_with_expanded_registration_dates.loc[
    df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < df_car_sales_with_expanded_registration_dates['Rok_produkcji'].astype(float)
][['Rok_produkcji', 'Rok_pierwszej_rejestracji']]

,Rok_produkcji,Rok_pierwszej_rejestracji
ID,,
822,1998.0,1997
1089,2011.0,2010
1424,2010.0,2009
1727,2006.0,0012
1890,2016.0,0017
...,...,...
132594,2007.0,2006
132905,2006.0,0019
133581,2011.0,2010


In [2090]:
# Sprawdzanie zcy Rok pierwszej rejestracji zawsze zawiera cyfry
mask = df_car_sales_with_expanded_registration_dates.loc[df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].str.contains('[\\d]') == False].index
df_car_sales_with_expanded_registration_dates.loc[mask]['Rok_pierwszej_rejestracji']


Series([], Name: Rok_pierwszej_rejestracji, dtype: object)

In [2091]:
df_car_sales_with_expanded_registration_dates.loc[df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].str.contains('[0-9]{4}') == False]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,


In [2092]:
test_data = df_car_sales_with_expanded_registration_dates

In [2093]:
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,
822,1998.0,28,05,1997
1089,2011.0,10,10,2010
1424,2010.0,12,12,2009
1727,2006.0,20,06,0012
1890,2016.0,10,08,0017
...,...,...,...,...
132594,2007.0,28,12,2006
132905,2006.0,07,06,0019
133581,2011.0,20,06,2010


In [2094]:
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,
822,1998.0,28,05,1997
1089,2011.0,10,10,2010
1424,2010.0,12,12,2009
1727,2006.0,20,06,0012
1890,2016.0,10,08,0017
...,...,...,...,...
132594,2007.0,28,12,2006
132905,2006.0,07,06,0019
133581,2011.0,20,06,2010


In [2095]:
test_data.iloc[821]

Rok_produkcji                    1998.0
Dzien_pierwszej_rejestracji          28
Miesiac_pierwszej_rejestracji        05
Rok_pierwszej_rejestracji          1997
Name: 822, dtype: object

In [2096]:
# Iterowanie po indeksach Rok_pierwszej_rejestracji i zastępowanie niewłaśiwyh wartości
for i in range(4):
    # Warunek
    mask = (
        df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].notnull()
        & df_car_sales_with_expanded_registration_dates['Rok_produkcji'].notnull()
        & (
            (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'])
            | (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) > 2021)
        )
    )

    # Sprawdzenie, czy są wiersze spełniające warunek
    if not df_car_sales_with_expanded_registration_dates[mask].empty:
        # Pobranie indeksów pasujących wierszy
        matching_indices = df_car_sales_with_expanded_registration_dates[mask].index

        # Iteracja przez pasujące wiersze
        for idx in matching_indices:
            # Pobranie roku produkcji i konwersja na string
            production_year = str(int(df_car_sales_with_expanded_registration_dates.loc[idx, 'Rok_produkcji']))
            # Pobranie pierwszej cyfry roku produkcji
            production_year_digit = production_year[i] if i < len(production_year) else ''

            # Pobranie roku rejestracji
            registration_year = test_data.loc[idx, 'Rok_pierwszej_rejestracji']
            # Podział roku rejestracji na głowę i ogon
            registration_year_head = registration_year[:i]
            registration_year_tail = registration_year[i + 1:]

            # Utworzenie nowej wartości roku rejestracji
            new_registration_year = registration_year_head + production_year_digit + registration_year_tail

            # Aktualizacja wartości w DataFrame
            test_data.at[idx, 'Rok_pierwszej_rejestracji'] = new_registration_year

# Sprawdzanie czy dalej występują niepoprawne wartości
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,


In [2097]:
# df_car_sales.loc[:, ["Rok_pierwszej_rejestracji", "Miesiac_pierwszej_rejestracji"]] = test_data[:, ["Rok_pierwszej_rejestracji", "Miesiac_pierwszej_rejestracji"]]

In [2098]:
# # Uzupełnij brakujące wartości w kolumnie "Rok_pierwszej_rejestracji" na podstawie kolumny "Rok_produkcji"
# df_car_sales.loc[df_car_sales["Rok_pierwszej_rejestracji"].isnull(), "Rok_pierwszej_rejestracji"] = df_car_sales.loc[df_car_sales["Rok_pierwszej_rejestracji"].isnull(), "Rok_produkcji"].apply(lambda x: x + 2)
# print(1)
# # Ogranicz wartości w kolumnie "Rok_pierwszej_rejestracji" do maksymalnie 2021
# df_car_sales.loc[df_car_sales["Rok_pierwszej_rejestracji"] > 2021, ["Rok_pierwszej_rejestracji"]] = 2021
# print(1)
#
# # Wyświetl kolumnę "Rok_pierwszej_rejestracji"
# df_car_sales["Rok_pierwszej_rejestracji"]
# print(1)

## Merge oczyszczony rok i miesiac pierwszej rejestracji z danymi z df_car_sales

In [2099]:
df_car_sales['Rok_pierwszej_rejestracji'] = df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji']
df_car_sales['Miesiac_pierwszej_rejestracji']= df_car_sales_with_expanded_registration_dates['Miesiac_pierwszej_rejestracji']
df_car_sales.drop(columns=['Data_pierwszej_rejestracji'], inplace=True)
df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,...,Skrzynia_biegow,Typ_nadwozia,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,PLN,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,...,Manual,minivan,blue,NaN,NaN,2021-04-28,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN
2,25900.0,PLN,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,...,Manual,station_wagon,silver,NaN,NaN,2021-05-04,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2010,06
3,35900.0,PLN,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,...,Manual,minivan,white,Denmark,NaN,2021-05-03,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ...",NaN,NaN
4,5999.0,PLN,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,...,Manual,compact,blue,NaN,NaN,2021-05-02,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air...",2007,11
5,44800.0,PLN,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,...,Manual,NaN,other,Poland,1.0,2021-05-02,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2013,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,PLN,Used,Opel,Astra,<NA>,K (2015-),2018.0,136931.0,150.0,...,Manual,compact,silver,Poland,1.0,NaT,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)","['ABS', 'ASR (traction control)', 'On-board co...",2018,08
135394,269855.0,PLN,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,W447 (2014-),2021.0,8.0,237.0,...,Automatic,minivan,white,NaN,NaN,2021-05-01,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)","['ABS', 'Electrically adjustable mirrors', 'Pa...",NaN,NaN
135395,21900.0,PLN,Used,<NA>,Zafira,<NA>,B (2005-2014),NaN,179000.0,120.0,...,Manual,minivan,black,NaN,NaN,2021-05-01,"WALERIANY 30 A - 96-330 Waleriany, żyrardowski...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN


In [2100]:
df_car_sales.loc[:, "Rok_publikacji_oferty"] = df_car_sales["Data_publikacji_oferty"].dt.year
df_car_sales.loc[:, "Miesiac_publikacji_oferty"] = df_car_sales["Data_publikacji_oferty"].dt.month
# Zmiana typu Rok_pierwszej_rejestracji i Miesiac_pierwszej_rejestracji
df_car_sales.loc[:, "Rok_pierwszej_rejestracji"] = df_car_sales.loc[:, "Rok_pierwszej_rejestracji"].astype('float')
df_car_sales.loc[:, "Miesiac_pierwszej_rejestracji"] = df_car_sales.loc[:, "Miesiac_pierwszej_rejestracji"].astype('float')

# Ujednolicenie nazw marek

In [2101]:
df_car_sales["Marka_pojazdu"]

ID
1               Renault
2               Renault
3                  Opel
4                  Ford
5                Toyota
              ...      
135393             Opel
135394    Mercedes-Benz
135395             <NA>
135396          Renault
135397             Opel
Name: Marka_pojazdu, Length: 135397, dtype: string

# Czyszczenie 'Generacja_pojazdu' i 'Model_pojazdu'

In [2102]:
car_gen = df_car_sales[['Generacja_pojazdu', 'Wersja_pojazdu']]
car_gen

,Generacja_pojazdu,Wersja_pojazdu
ID,,
1,<NA>,Gr 2.0T 16V Expression
2,III (2008-2016),1.6 16V 110
3,C (2011-2019),Tourer 1.6 CDTI ecoFLEX Start/Stop
4,Mk2 (2004-2011),1.6 TDCi FX Silver / Silver X
5,III (2009-),1.8
...,...,...
135393,K (2015-),<NA>
135394,W447 (2014-),124 CDI Tourer Lang
135395,B (2005-2014),<NA>


In [2103]:
# Obsługa brakujących danych
car_gen.loc[car_gen['Generacja_pojazdu'].isnull()] = 'Nieznana'
car_gen.loc[car_gen['Wersja_pojazdu'].isnull()] = 'Nieznana'
car_gen


,Generacja_pojazdu,Wersja_pojazdu
ID,,
1,Nieznana,Nieznana
2,III (2008-2016),1.6 16V 110
3,C (2011-2019),Tourer 1.6 CDTI ecoFLEX Start/Stop
4,Mk2 (2004-2011),1.6 TDCi FX Silver / Silver X
5,III (2009-),1.8
...,...,...
135393,Nieznana,Nieznana
135394,W447 (2014-),124 CDI Tourer Lang
135395,Nieznana,Nieznana


In [2104]:
# Wyodrębnienie generacji i lat produkcji
tmp_data = pd.DataFrame()
tmp_data['Nazwa_generacji'] = car_gen['Generacja_pojazdu'].str.extract(r'([^(]+)\s', expand=True)
tmp_data['Rok_startu_produkcji'] = car_gen['Generacja_pojazdu'].str.extract(r'\s\((\d*)-', expand=True)
tmp_data['Rok_konca_produkcji'] = car_gen['Generacja_pojazdu'].str.extract(r'-(\d*)\)', expand=True)
# car_gen.loc[:,["Generacja_pojazdu"]] = tmp_data[0]
tmp_data


,Nazwa_generacji,Rok_startu_produkcji,Rok_konca_produkcji
ID,,,
1,<NA>,<NA>,<NA>
2,III,2008,2016
3,C,2011,2019
4,Mk2,2004,2011
5,III,2009,
...,...,...,...
135393,<NA>,<NA>,<NA>
135394,W447,2014,
135395,<NA>,<NA>,<NA>


In [2105]:
df_car_sales = pd.concat([df_car_sales, tmp_data], ignore_index=False, axis=1)
# df_car_sales.iloc[:, "Generacja_pojazdu"] = tmp_data['Generacja_pojazdu']
# df_car_sales.iloc[:, "Rok_startu_produkcji"] = tmp_data['rok_startu_produkcji']
# df_car_sales.iloc[:, "Rok_konca_produkcji"] = tmp_data['rok_konca_produkcji']
df_car_sales.drop(columns=['Generacja_pojazdu'], inplace=True)
df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_publikacji_oferty,Miesiac_publikacji_oferty,Nazwa_generacji,Rok_startu_produkcji,Rok_konca_produkcji
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,PLN,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,2005.0,213000.0,170.0,1998.0,...,2021-04-28,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN,2021.0,4.0,<NA>,<NA>,<NA>
2,25900.0,PLN,Used,Renault,Megane,1.6 16V 110,2010.0,117089.0,110.0,1598.0,...,2021-05-04,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2010.0,6.0,2021.0,5.0,III,2008,2016
3,35900.0,PLN,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,2015.0,115600.0,136.0,1598.0,...,2021-05-03,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ...",NaN,NaN,2021.0,5.0,C,2011,2019
4,5999.0,PLN,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,2007.0,218000.0,90.0,1560.0,...,2021-05-02,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air...",2007.0,11.0,2021.0,5.0,Mk2,2004,2011
5,44800.0,PLN,Used,Toyota,Avensis,1.8,2013.0,NaN,NaN,1798.0,...,2021-05-02,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2013.0,5.0,2021.0,5.0,III,2009,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,PLN,Used,Opel,Astra,<NA>,2018.0,136931.0,150.0,1598.0,...,NaT,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)","['ABS', 'ASR (traction control)', 'On-board co...",2018.0,8.0,NaN,NaN,<NA>,<NA>,<NA>
135394,269855.0,PLN,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,2021.0,8.0,237.0,1950.0,...,2021-05-01,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)","['ABS', 'Electrically adjustable mirrors', 'Pa...",NaN,NaN,2021.0,5.0,W447,2014,
135395,21900.0,PLN,Used,<NA>,Zafira,<NA>,NaN,179000.0,120.0,1700.0,...,2021-05-01,"WALERIANY 30 A - 96-330 Waleriany, żyrardowski...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN,2021.0,5.0,<NA>,<NA>,<NA>


In [2106]:
# Podział 'Wersja_pojazdu'
tmp_data = car_gen['Wersja_pojazdu'].str.extract(r'\s*(\d+\.\d+\s*\w*)', expand=True)
df_car_sales.loc[:, "Typ_silnika"] = tmp_data[0]
df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_publikacji_oferty,Miesiac_publikacji_oferty,Nazwa_generacji,Rok_startu_produkcji,Rok_konca_produkcji,Typ_silnika
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,PLN,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,2005.0,213000.0,170.0,1998.0,...,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN,2021.0,4.0,<NA>,<NA>,<NA>,<NA>
2,25900.0,PLN,Used,Renault,Megane,1.6 16V 110,2010.0,117089.0,110.0,1598.0,...,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2010.0,6.0,2021.0,5.0,III,2008,2016,1.6 16V
3,35900.0,PLN,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,2015.0,115600.0,136.0,1598.0,...,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ...",NaN,NaN,2021.0,5.0,C,2011,2019,1.6 CDTI
4,5999.0,PLN,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,2007.0,218000.0,90.0,1560.0,...,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air...",2007.0,11.0,2021.0,5.0,Mk2,2004,2011,1.6 TDCi
5,44800.0,PLN,Used,Toyota,Avensis,1.8,2013.0,NaN,NaN,1798.0,...,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2013.0,5.0,2021.0,5.0,III,2009,,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,PLN,Used,Opel,Astra,<NA>,2018.0,136931.0,150.0,1598.0,...,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)","['ABS', 'ASR (traction control)', 'On-board co...",2018.0,8.0,NaN,NaN,<NA>,<NA>,<NA>,<NA>
135394,269855.0,PLN,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,2021.0,8.0,237.0,1950.0,...,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)","['ABS', 'Electrically adjustable mirrors', 'Pa...",NaN,NaN,2021.0,5.0,W447,2014,,<NA>
135395,21900.0,PLN,Used,<NA>,Zafira,<NA>,NaN,179000.0,120.0,1700.0,...,"WALERIANY 30 A - 96-330 Waleriany, żyrardowski...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN,2021.0,5.0,<NA>,<NA>,<NA>,<NA>


# Uzupełnianie braków z "Rok_Produkcji"

In [2107]:
# Grupowanie po marce i modelu, uzupełnianie brakujących lat produkcji
def fill_production_years(group):
    # Uzupełnij brakujące lata początkowe najwcześniejszą znaną wartością w grupie
    start_year = group['Rok_startu_produkcji'].min()
    group['Rok_startu_produkcji'] = group['Rok_startu_produkcji'].fillna(start_year)

    # Uzupełnij brakujące lata końcowe
    end_year = group['Rok_konca_produkcji'].max()
    group['Rok_konca_produkcji'] = group['Rok_konca_produkcji'].fillna(end_year)

    return group

# Zastosuj funkcję do grup
df_filled = df_car_sales.groupby(['Marka_pojazdu', 'Model_pojazdu'], group_keys=False).apply(fill_production_years)

C:\Users\vipcy\AppData\Local\Temp\ipykernel_18164\2648067611.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled = df_car_sales.groupby(['Marka_pojazdu', 'Model_pojazdu'], group_keys=False).apply(fill_production_years)


In [2108]:
df_filled.loc[df_filled["Rok_produkcji"] != ""].info()

<class 'pandas.core.frame.DataFrame'>
Index: 128814 entries, 1 to 135397
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Cena                           128814 non-null  float64       
 1   Waluta                         125600 non-null  category      
 2   Stan                           125653 non-null  category      
 3   Marka_pojazdu                  128814 non-null  string        
 4   Model_pojazdu                  128814 non-null  string        
 5   Wersja_pojazdu                 83128 non-null   string        
 6   Rok_produkcji                  125572 non-null  float64       
 7   Przebieg_km                    124973 non-null  float64       
 8   Moc_KM                         125268 non-null  float64       
 9   Pojemnosc_cm3                  124343 non-null  float64       
 10  Rodzaj_paliwa                  125576 non-null  category      
 11  Naped

In [2109]:
df_car_sales.loc[:, "Rok_produkcji"] = df_filled["Rok_produkcji"]
df_car_sales.loc[:, "Rok_startu_produkcji"] = df_filled["Rok_startu_produkcji"]
df_car_sales.loc[:, "Rok_konca_produkcji"] = df_filled["Rok_konca_produkcji"]

In [2110]:
print(df_car_sales[["Rok_produkcji", "Rok_startu_produkcji", "Rok_konca_produkcji"]].info())
df_car_sales["Rok_startu_produkcji"] = df_car_sales["Rok_startu_produkcji"].astype(float)
df_car_sales["Rok_konca_produkcji"] = df_car_sales["Rok_konca_produkcji"].replace("", np.nan).astype(float)


mask = df_car_sales.loc[df_car_sales['Rok_startu_produkcji'].notnull() & df_car_sales['Rok_konca_produkcji'].notnull()].index
mean_time_of_production = round((df_car_sales.loc[mask, 'Rok_konca_produkcji'] - df_car_sales.loc[mask, 'Rok_startu_produkcji'] ).mean())

mask = df_car_sales.loc[df_car_sales['Rok_startu_produkcji'].notnull() & df_car_sales['Rok_konca_produkcji'].isnull()].index
df_car_sales.loc[mask, 'Rok_konca_produkcji'] = df_car_sales.loc[mask, 'Rok_startu_produkcji'] + mean_time_of_production



mask = df_car_sales.loc[df_car_sales['Rok_startu_produkcji'].isnull() & df_car_sales['Rok_konca_produkcji'].notnull()].index
df_car_sales.loc[mask, 'Rok_startu_produkcji'] = df_car_sales.loc[mask, 'Rok_konca_produkcji'] - mean_time_of_production

mask = df_car_sales.loc[ df_car_sales['Rok_produkcji'].notnull() & (df_car_sales['Rok_startu_produkcji'].isnull() | df_car_sales['Rok_konca_produkcji'].isnull())].index
df_car_sales.loc[mask, 'Rok_startu_produkcji'] = round(df_car_sales.loc[mask, 'Rok_produkcji'] - (mean_time_of_production /2))
df_car_sales.loc[mask, 'Rok_konca_produkcji'] = round(df_car_sales.loc[mask, 'Rok_produkcji'] + (mean_time_of_production /2))

end_of_production_after_2021 = df_car_sales.loc[df_car_sales['Rok_konca_produkcji'] > 2021].index
df_car_sales.loc[end_of_production_after_2021, 'Rok_konca_produkcji'] = 2021
df_car_sales.iloc[:, :20].info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 3 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Rok_produkcji         125572 non-null  float64
 1   Rok_startu_produkcji  93896 non-null   string 
 2   Rok_konca_produkcji   93896 non-null   string 
dtypes: float64(1), string(2)
memory usage: 8.2 MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Cena                    135397 non-null  float64       
 1   Waluta                  132021 non-null  category      
 2   Stan                    132075 non-null  category      
 3   Marka_pojazdu           132046 non-null  string        
 4   Model_pojazdu           132088 non-null  string        
 5   Wersja_pojazdu          87336 non-null   

# Konwersja EUR -> PLN

In [2111]:
# Slekcja dadnych z 'Waluta' == 'EUR'
# euro_to_pln = df_car_sales.loc[df_car_sales['Waluta'] == "EUR"]
# euro_to_pln
df_car_sales['Waluta'] = df_car_sales['Waluta'].cat.add_categories(['NotKnown']).fillna('NotKnown')
df_car_sales['Waluta'] = df_car_sales['Waluta'].map({'PLN': 1, 'EUR': 0, 'NotKnown': -1}).astype(int)

In [2112]:
df_car_sales["Waluta"].info()

<class 'pandas.core.series.Series'>
Index: 135397 entries, 1 to 135397
Series name: Waluta
Non-Null Count   Dtype
--------------   -----
135397 non-null  int32
dtypes: int32(1)
memory usage: 5.6 MB


In [2113]:
# import requests
# import datetime
# def get_exchange_rate(date: datetime, currency="EUR"):
#     """
#     Pobiera kurs EUR/PLN dla podanej daty.
#     :param date: Data w formacie 'YYYY-MM-DD'.
#     :param currency: Kod waluty (domyślnie 'EUR').
#     :return: Kurs wymiany lub None, jeśli dane nie są dostępne.
#     """
#     url = f'https://api.nbp.pl/api/exchangerates/rates/a/{currency.lower()}/{date}/?format=json'
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Sprawdza, czy odpowiedź jest poprawna
#         data = response.json()
#         return data["rates"][0]["mid"]  # Kurs średni
#     except requests.exceptions.HTTPError as e:
#         # print(f"Błąd podczas pobierania danych: {e}")
#         return np.nan


In [2114]:
# # Selekcja unikalnych dat z 'Waluta' == 'EUR'
# unique_dates = euro_to_pln['Data_publikacji_oferty'].unique()
# unique_dates_df = pd.DataFrame(unique_dates, columns=["Date"])
# unique_dates_df

In [2115]:
# # Tworzenie słownika z wartościami wymiany
# dict_values = {}
# list_of_exchange_rates = []
# for i, index in enumerate(unique_dates.tolist()):
#     exchange_rate = get_exchange_rate(str(index).split(" ")[0], 'EUR')
#     dict_values[index] = (float(exchange_rate))
#     list_of_exchange_rates.append(exchange_rate)
# unique_dates_df['Exchange_rate'] = pd.Series(list_of_exchange_rates).bfill().ffill()
# unique_dates_df

In [2116]:
# df_car_sales.loc[df_car_sales["Waluta"] == "EUR", "Cena"] = df_car_sales.loc[df_car_sales["Waluta"] == "EUR", "Cena"] * 4.2

In [2117]:

# for row in unique_dates_df.itertuples():
#     mask = euro_to_pln.loc[euro_to_pln['Data_publikacji_oferty'] == row[1]].index
#     mask_len = len(mask)
#     exchange_rate_numpy = pd.Series(np.full(mask_len, row[2]))
#     df_car_sales.loc[mask, 'Cena'] = df_car_sales.loc[mask, 'Cena'] * row[2]
# df_car_sales.drop(columns=['Waluta'], inplace=True)
# df_car_sales.drop(columns=['Data_publikacji_oferty'], inplace=True)

df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_publikacji_oferty,Miesiac_publikacji_oferty,Nazwa_generacji,Rok_startu_produkcji,Rok_konca_produkcji,Typ_silnika
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,1,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,2005.0,213000.0,170.0,1998.0,...,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN,2021.0,4.0,<NA>,1999.0,2011.0,<NA>
2,25900.0,1,Used,Renault,Megane,1.6 16V 110,2010.0,117089.0,110.0,1598.0,...,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2010.0,6.0,2021.0,5.0,III,2008.0,2016.0,1.6 16V
3,35900.0,1,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,2015.0,115600.0,136.0,1598.0,...,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ...",NaN,NaN,2021.0,5.0,C,2011.0,2019.0,1.6 CDTI
4,5999.0,1,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,2007.0,218000.0,90.0,1560.0,...,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air...",2007.0,11.0,2021.0,5.0,Mk2,2004.0,2011.0,1.6 TDCi
5,44800.0,1,Used,Toyota,Avensis,1.8,2013.0,NaN,NaN,1798.0,...,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2013.0,5.0,2021.0,5.0,III,2009.0,2021.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,1,Used,Opel,Astra,<NA>,2018.0,136931.0,150.0,1598.0,...,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)","['ABS', 'ASR (traction control)', 'On-board co...",2018.0,8.0,NaN,NaN,<NA>,1991.0,2015.0,<NA>
135394,269855.0,1,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,2021.0,8.0,237.0,1950.0,...,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)","['ABS', 'Electrically adjustable mirrors', 'Pa...",NaN,NaN,2021.0,5.0,W447,2014.0,2021.0,<NA>
135395,21900.0,1,Used,<NA>,Zafira,<NA>,NaN,179000.0,120.0,1700.0,...,"WALERIANY 30 A - 96-330 Waleriany, żyrardowski...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN,2021.0,5.0,<NA>,NaN,NaN,<NA>


# 'Wyposazenie'

In [2118]:
# Wylistowanie wszystkich elementów z atrybutu 'Wyposazenie'
equipment_list = df_car_sales['Wyposazenie'].dropna().str.strip("[]").str.replace("'", "").str.split(", ")
# Zredukowanie listy do set'a z unikalnymi wartościami
unique_equipment = set([item for sublist in equipment_list for item in sublist if item])

# Stworzenie One Hot encoded atrybutu dla każdego elementu
for item in unique_equipment:
    df_car_sales[item] = df_car_sales['Wyposazenie'].str.contains(item, regex=False)
df_car_sales.drop(columns=['Wyposazenie'], inplace=True)
df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,SD socket,CD changer,Rear view camera,Airbag protecting the knees,Lane assistant,Isofix,Automatic air conditioning,Front side airbags,Central locking,Blind spot sensor
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,1,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,2005.0,213000.0,170.0,1998.0,...,False,False,False,False,False,True,False,False,True,False
2,25900.0,1,Used,Renault,Megane,1.6 16V 110,2010.0,117089.0,110.0,1598.0,...,False,False,False,False,False,True,False,False,True,False
3,35900.0,1,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,2015.0,115600.0,136.0,1598.0,...,True,False,False,False,False,True,True,False,True,False
4,5999.0,1,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,2007.0,218000.0,90.0,1560.0,...,False,False,False,False,False,False,False,False,True,False
5,44800.0,1,Used,Toyota,Avensis,1.8,2013.0,NaN,NaN,1798.0,...,False,False,False,False,False,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,1,Used,Opel,Astra,<NA>,2018.0,136931.0,150.0,1598.0,...,False,False,False,False,False,True,True,False,False,False
135394,269855.0,1,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,2021.0,8.0,237.0,1950.0,...,False,False,True,False,False,True,True,False,True,False
135395,21900.0,1,Used,<NA>,Zafira,<NA>,NaN,179000.0,120.0,1700.0,...,False,False,True,False,False,False,True,False,True,False


In [2119]:
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 97 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Cena                             135397 non-null  float64       
 1   Waluta                           135397 non-null  int32         
 2   Stan                             132075 non-null  category      
 3   Marka_pojazdu                    132046 non-null  string        
 4   Model_pojazdu                    132088 non-null  string        
 5   Wersja_pojazdu                   87336 non-null   string        
 6   Rok_produkcji                    125572 non-null  float64       
 7   Przebieg_km                      131394 non-null  float64       
 8   Moc_KM                           131664 non-null  float64       
 9   Pojemnosc_cm3                    130711 non-null  float64       
 10  Rodzaj_paliwa                    131987 non-null 

# Lokalizacja_oferty

In [2120]:
# df_car_sales.loc[ df_car_sales['Lokalizacja_oferty'].str.contains('[\\d]{2}-[\\d]{3}', regex=True) == False]['Lokalizacja_oferty']
# df_car_sales.loc[df_car_sales['Lokalizacja_oferty'].str.contains('([\\d]{2}\\w*-?\\w*[\\d]{3}\\s\\w*\\s*),', regex=True)]['Lokalizacja_oferty']
# df_car_sales.loc[ df_car_sales['Lokalizacja_oferty'].str.contains('[(Polska]', regex=True) == False &  df_car_sales['Lokalizacja_oferty'].str.contains('[(]', regex=True) ]['Lokalizacja_oferty']
df = df_car_sales.loc[df_car_sales['Lokalizacja_oferty'].str.contains("\\([\\w*]")]['Lokalizacja_oferty']
df.loc[df.str.contains("Polska") == False]

ID
2860      grensstraat - 2243 Antwerpia, antwerpia, Belgi...
2877      Calle Assagador De Ricardo, 57 - 03503 Madrid,...
7505      OC Centro, Sjízdná 4, OSTRAVA - 722 00 Novy Ji...
9868      grensstraat - 2243 Antwerpia, antwerpia, Belgi...
10015     Calle Assagador De Ricardo, 57 - 03503 Madrid,...
                                ...                        
122378    Calle Assagador De Ricardo, 57 - 03503 Madrid,...
126627    K mostu 393 Stara ves nad Ondřejnici - 73923 N...
129702    K mostu 393 Stara ves nad Ondřejnici - 73923 N...
131998    PS DAmunt 7 - 08024 Madrid, Comunidad de Madri...
133348    OC Centro, Sjízdná 4, OSTRAVA - 722 00 Novy Ji...
Name: Lokalizacja_oferty, Length: 65, dtype: string

In [2121]:
localization_df = pd.DataFrame()
localization_df['Lokalizacja_oferty'] = df_car_sales['Lokalizacja_oferty']
localization_df["Kod_pocztowy"] = df_car_sales["Lokalizacja_oferty"].str.extract('([\\d]{2}\\w*-?\\w*[\\d]{3}s*)')
localization_df["Miejscowosc"] = df_car_sales["Lokalizacja_oferty"].str.extract('[\\d]{2}\\w*-?\\w*[\\d]{3}\\s(\\w*\\s*\\w*)')
localization_df.loc[localization_df["Miejscowosc"].isnull()] = localization_df.loc[localization_df["Miejscowosc"].isnull()]["Miejscowosc"].str.extract('^(\\w*),')
localization_df["Ulica"] = df_car_sales["Lokalizacja_oferty"].str.extract('(ul\\.?\\s*?[A-Za-ząćęłńóśźżĄĆĘŁŃÓŚŹŻ\\s]*)')
localization_df.loc[localization_df["Lokalizacja_oferty"].notnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica
ID,,,,
1,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...",99-300,Kutno,<NA>
2,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)",41-407,Imielin,ul. Wiosenna
7,Ul. Buforowa 125 / Hala nr 23A - 52-131 Wrocła...,52-131,Wrocław,<NA>
11,"ul. Radzymińska 78 - 03-752 Warszawa, Praga-Pó...",03-752,Warszawa,ul. Radzymińska
12,"Zatorze 1 - 02-852 Warszawa, Mazowieckie (Polska)",02-852,Warszawa,<NA>
...,...,...,...,...
135392,"Kasprzaka 1 - 66-400 Gorzów Wielkopolski, Lubu...",66-400,Gorzów Wielkopolski,<NA>
135393,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)",53-012,Wrocław,<NA>
135394,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)",26-600,Radom,<NA>


In [2122]:
cities_df = pd.read_csv('woj/wojewodztwa_miasta.csv', sep=';', index_col=0)
unique_wojewodztwa = cities_df["nazwa województwa"].unique()
cities_df

,nazwa miasta,nazwa województwa
id,,
1,Bolesławiec,DOLNOŚLĄSKIE
2,Nowogrodziec,DOLNOŚLĄSKIE
3,Bielawa,DOLNOŚLĄSKIE
4,Dzierżoniów,DOLNOŚLĄSKIE
5,Pieszyce,DOLNOŚLĄSKIE
...,...,...
914,Resko,ZACHODNIOPOMORSKIE
915,Węgorzyno,ZACHODNIOPOMORSKIE
916,Koszalin,ZACHODNIOPOMORSKIE


In [2123]:
localization_df["Wojewodztwo"] = pd.Series(dtype='str')
wojewodztwa = cities_df.groupby(by='nazwa województwa').count().index.tolist()
for index in wojewodztwa:
    # print(index)
    mask = localization_df.loc[localization_df["Lokalizacja_oferty"].str.upper().str.contains(index) & localization_df["Lokalizacja_oferty"].notnull()].index
    localization_df.loc[mask, 'Wojewodztwo'] = index
    # print(localization_df.iloc[mask])

    # localization_df[mask, ["Wojewodztwo"]] = index
localization_df.loc[localization_df["Wojewodztwo"].notnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica,Wojewodztwo
ID,,,,,
1,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...",99-300,Kutno,<NA>,ŁÓDZKIE
12,"Zatorze 1 - 02-852 Warszawa, Mazowieckie (Polska)",02-852,Warszawa,<NA>,MAZOWIECKIE
14,"Harcerska, 64 - 43-100 Tychy, Śląskie (Polska)",43-100,Tychy,<NA>,ŚLĄSKIE
18,"Obornicka 72-74 - 62-002 Suchy Las, poznański,...",62-002,Suchy Las,<NA>,WIELKOPOLSKIE
19,"ul. Stalowa 16 - 41-506 Chorzów, Śląskie (Polska)",41-506,Chorzów,ul. Stalowa,ŚLĄSKIE
...,...,...,...,...,...
135386,"ul. Stalowa 16 - 41-506 Chorzów, Śląskie (Polska)",41-506,Chorzów,ul. Stalowa,ŚLĄSKIE
135392,"Kasprzaka 1 - 66-400 Gorzów Wielkopolski, Lubu...",66-400,Gorzów Wielkopolski,<NA>,LUBUSKIE
135394,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)",26-600,Radom,<NA>,MAZOWIECKIE


In [2124]:
localization_df["Miejscowosc2"] = pd.Series(dtype='str')
cities = cities_df.groupby(by='nazwa miasta').count().index.tolist() + ["Warszawa"]
for index in cities:
    mask = localization_df.loc[localization_df["Lokalizacja_oferty"].str.upper().str.contains(index.upper()) & localization_df["Lokalizacja_oferty"].notnull()].index
    localization_df.loc[mask, 'Miejscowosc2'] = index


localization_df.loc[localization_df["Miejscowosc2"].isnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica,Wojewodztwo,Miejscowosc2
ID,,,,,,
3,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,<NA>,<NA>,<NA>,<NA>,NaN,NaN
5,<NA>,<NA>,<NA>,<NA>,NaN,NaN
6,<NA>,<NA>,<NA>,<NA>,NaN,NaN
8,<NA>,<NA>,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...
135385,<NA>,<NA>,<NA>,<NA>,NaN,NaN
135388,<NA>,<NA>,<NA>,<NA>,NaN,NaN
135389,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [2125]:
localization_df.loc[localization_df["Lokalizacja_oferty"].isnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica,Wojewodztwo,Miejscowosc2
ID,,,,,,
3,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,<NA>,<NA>,<NA>,<NA>,NaN,NaN
5,<NA>,<NA>,<NA>,<NA>,NaN,NaN
6,<NA>,<NA>,<NA>,<NA>,NaN,NaN
8,<NA>,<NA>,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...
135384,<NA>,<NA>,<NA>,<NA>,NaN,NaN
135385,<NA>,<NA>,<NA>,<NA>,NaN,NaN
135388,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [2126]:
mask = localization_df.loc[localization_df["Miejscowosc"].notnull()].index
localization_df.loc[mask, "Miejscowosc2"] = localization_df.loc[mask, "Miejscowosc"]
localization_df.loc[localization_df["Miejscowosc2"].notnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica,Wojewodztwo,Miejscowosc2
ID,,,,,,
1,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...",99-300,Kutno,<NA>,ŁÓDZKIE,Kutno
2,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)",41-407,Imielin,ul. Wiosenna,NaN,Imielin
7,Ul. Buforowa 125 / Hala nr 23A - 52-131 Wrocła...,52-131,Wrocław,<NA>,NaN,Wrocław
11,"ul. Radzymińska 78 - 03-752 Warszawa, Praga-Pó...",03-752,Warszawa,ul. Radzymińska,NaN,Warszawa
12,"Zatorze 1 - 02-852 Warszawa, Mazowieckie (Polska)",02-852,Warszawa,<NA>,MAZOWIECKIE,Warszawa
...,...,...,...,...,...,...
135392,"Kasprzaka 1 - 66-400 Gorzów Wielkopolski, Lubu...",66-400,Gorzów Wielkopolski,<NA>,LUBUSKIE,Gorzów Wielkopolski
135393,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)",53-012,Wrocław,<NA>,NaN,Wrocław
135394,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)",26-600,Radom,<NA>,MAZOWIECKIE,Radom


In [2127]:
'''Uzupełnianie Wojewodztw gdzie miasto jest znane na bazie cities_df'''
mask = localization_df.loc[localization_df["Wojewodztwo"].isnull() & localization_df["Miejscowosc2"].notnull()].index
localization_df.loc[localization_df["Miejscowosc2"].str.replace(" ", "") == "Warszawa", "Wojewodztwo"] = "MAZOWIECKIE"
for city in localization_df.loc[mask]["Miejscowosc2"].unique():
    city = city.replace(" ", "")
    if city.upper() in cities_df["nazwa miasta"].str.replace(" ", "").str.upper().tolist():
        if cities_df.loc[cities_df["nazwa miasta"].str.replace(" ", "").str.upper() == city.upper()]["nazwa województwa"].shape[0] == 0:
            continue
        localization_df.loc[localization_df["Miejscowosc2"].str.replace(" ", "").str.upper() == city.upper(), "Wojewodztwo"] = cities_df.loc[cities_df["nazwa miasta"].str.replace(" ", "").str.upper() == city.upper()]["nazwa województwa"].iloc[0]

localization_df.loc[(localization_df["Miejscowosc2"] == ""), ['Lokalizacja_oferty','Wojewodztwo', "Miejscowosc2"]] = np.nan

In [2128]:
df_car_sales["Wojewodztwo"] = localization_df["Wojewodztwo"]
df_car_sales["Miejscowosc"] = localization_df["Miejscowosc2"]
# df_car_sales[["Lokalizacja_oferty", "Wojewodztwo", "Miejscowosc"]]

In [2129]:
df_car_sales.drop(columns="Lokalizacja_oferty", inplace=True)

#

In [2130]:
# Zmiana typu roku startu i końca produkcji na typ float
df_car_sales.loc[df_car_sales["Rok_startu_produkcji"] == "" , ["Rok_startu_produkcji"]] = np.nan
df_car_sales["Rok_startu_produkcji"] = df_car_sales["Rok_startu_produkcji"].astype('float64')
df_car_sales.loc[df_car_sales["Rok_konca_produkcji"] == "" , ["Rok_konca_produkcji"]] = np.nan
df_car_sales["Rok_konca_produkcji"] = df_car_sales["Rok_konca_produkcji"].astype('float64')

In [2131]:
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 98 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Cena                             135397 non-null  float64       
 1   Waluta                           135397 non-null  int32         
 2   Stan                             132075 non-null  category      
 3   Marka_pojazdu                    132046 non-null  string        
 4   Model_pojazdu                    132088 non-null  string        
 5   Wersja_pojazdu                   87336 non-null   string        
 6   Rok_produkcji                    125572 non-null  float64       
 7   Przebieg_km                      131394 non-null  float64       
 8   Moc_KM                           131664 non-null  float64       
 9   Pojemnosc_cm3                    130711 non-null  float64       
 10  Rodzaj_paliwa                    131987 non-null 

In [2132]:
df_car_sales["Wojewodztwo"].unique()

array(['ŁÓDZKIE', 'ŚLĄSKIE', nan, 'DOLNOŚLĄSKIE', 'MAZOWIECKIE',
       'WIELKOPOLSKIE', 'PODKARPACKIE', 'OPOLSKIE', 'MAŁOPOLSKIE',
       'POMORSKIE', 'WARMIŃSKO-MAZURSKIE', 'LUBUSKIE',
       'ZACHODNIOPOMORSKIE', 'LUBELSKIE', 'PODLASKIE', 'ŚWIĘTOKRZYSKIE'],
      dtype=object)

In [2133]:
new_df = pd.concat([pd.get_dummies(df_car_sales["Stan"]), pd.get_dummies(df_car_sales["Naped"]),
                    pd.get_dummies(df_car_sales["Rodzaj_paliwa"]), pd.get_dummies(df_car_sales["Skrzynia_biegow"]), pd.get_dummies(df_car_sales["Typ_nadwozia"])], axis=1)
df_car_sales = pd.concat([df_car_sales, new_df], axis=1)
df_car_sales.drop(columns=["Naped", "Stan", "Rodzaj_paliwa", "Skrzynia_biegow", "Typ_nadwozia"], inplace=True)
df_car_sales

,Cena,Waluta,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,Kolor,...,Manual,SUV,city_cars,compact,convertible,coupe,minivan,sedan,small_cars,station_wagon
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,1,Renault,Grand Espace,Gr 2.0T 16V Expression,2005.0,213000.0,170.0,1998.0,blue,...,True,False,False,False,False,False,True,False,False,False
2,25900.0,1,Renault,Megane,1.6 16V 110,2010.0,117089.0,110.0,1598.0,silver,...,True,False,False,False,False,False,False,False,False,True
3,35900.0,1,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,2015.0,115600.0,136.0,1598.0,white,...,True,False,False,False,False,False,True,False,False,False
4,5999.0,1,Ford,Focus,1.6 TDCi FX Silver / Silver X,2007.0,218000.0,90.0,1560.0,blue,...,True,False,False,True,False,False,False,False,False,False
5,44800.0,1,Toyota,Avensis,1.8,2013.0,NaN,NaN,1798.0,other,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,1,Opel,Astra,<NA>,2018.0,136931.0,150.0,1598.0,silver,...,True,False,False,True,False,False,False,False,False,False
135394,269855.0,1,Mercedes-Benz,Vito,124 CDI Tourer Lang,2021.0,8.0,237.0,1950.0,white,...,False,False,False,False,False,False,True,False,False,False
135395,21900.0,1,<NA>,Zafira,<NA>,NaN,179000.0,120.0,1700.0,black,...,True,False,False,False,False,False,True,False,False,False


## One-Hot encoding dla głównych marek samochodów i tworzenie dodatkowej kolumny dla reszty
Pozostałe marki kodowane za pomocą znormalizowanej wartości wysokości średniej ceny tej marki

In [2134]:
less_common_brands_average_prices = {
'Chevrolet': 40334,
 'MINI': 68992,
 'Alfa Romeo': 71619,
 'Land Rover': 202786,
 'Porsche': 305444,
 'Jaguar': 122483,
 'Lexus': 112199,
 'Subaru': 51313,
 'Chrysler': 44328,
 'Dodge': 96488,
 'Saab': 15444,
 'Smart': 23700,
 'Infiniti': 73901,
 'Lancia': 21118,
 'SsangYong': 55350,
 'Cupra': 184737,
 'Maserati': 234269,
 'Cadillac': 109760,
 'Daihatsu': 9588,
 'Aixam': 27096,
 'DS Automobiles': 149120,
 'Daewoo': 5628,
 'Tesla': 219562,
 'Bentley': 694845,
 'Microcar': 23228,
 'Isuzu': 116011,
 'Inny': 31586,
 'Lincoln': 50886,
 'RAM': 279539,
 'Abarth': 68612,
 'Ferrari': 830073,
 'Rover': 9254,
 'Lada': 26231,
 'Polonez': 9074,
 'Ligier': 39966,
 'Buick': 62947,
 'Hummer': 105558,
 'MG': 38041,
 'Rolls-Royce': 1146686,
 'Trabant': 12812,
 'Lamborghini': 1025579,
 'McLaren': 1488635,
 'Aston Martin': 476573,
 'Pontiac': 57999,
 'Acura': 67311,
 'Austin': 18808,
 'Uaz': 77176,
 'Oldsmobile': 49179,
 'GMC': 59800,
 'Plymouth': 56833,
 'Warszawa': 50322,
 'Tata': 10444,
 'Gaz': 35475,
 'Wołga': 33914,
 'Triumph': 18357,
 'Żuk': 17271,
 'Tarpan': 20070,
 'Chatenet': 38180,
 'DKW': 53340,
 'Moskwicz': 8340,
 'Wartburg': 26293,
 'Zastava': 11550,
 'Alpine': 386919,
 'Mercury': 37333,
 'Casalini': 14466,
 'DFSK': 119310,
 'Syrena': 26300,
 'Iveco': 51867,
 'Lotus': 112167,
 'Vauxhall': 8900,
 'Tavria': 6450,
 'Grecav': 7500,
 'Nysa': 21450,
 'FAW': 132447,
 'Baic': 10000,
 'Autobianchi': 9900,
 'Vanderhall': 149900,
 'Zaporożec': 33333,
 'Saturn': 38900
                                     }

rest_brands = [
    'Suzuki', 'Mitsubishi', 'Jeep', 'Dacia', 'Chevrolet', 'MINI', 'Alfa Romeo', 'Land Rover', 'Porsche', 'Lexus', 'Jaguar', 'Subaru', 'Dodge', 'Chrysler', 'Saab', 'Infiniti', 'Smart', 'Lancia', 'Maserati', 'SsangYong', 'Aixam', 'Cadillac', 'Daewoo', 'Daihatsu', 'DS Automobiles', 'Cupra', 'Tesla', 'Bentley', 'Polonez', 'Isuzu', 'Lincoln', 'Ferrari', 'Microcar', 'Rover', 'Abarth', 'Ligier', 'Inny', 'RAM', 'Lada', 'Rolls-Royce', 'Trabant', 'Buick', 'Hummer', 'Pontiac', 'Oldsmobile', 'MG', 'Uaz', 'Aston Martin', 'GMC', 'Lamborghini', 'Warszawa', 'Syrena', 'Wołga', 'DFSK', 'Żuk', 'Austin', 'Maybach', 'Wartburg', 'Alpine', 'Iveco', 'Baic', 'Zaporożec', 'Tata', 'Moskwicz', 'Nysa', 'Zastava', 'Plymouth', 'Triumph', 'McLaren', 'Chatenet', 'Acura', 'Santana', 'Talbot', 'Scion', 'MAN', 'NSU', 'Gaz', 'Casalini', 'Tarpan', 'Lotus', 'Mercury', 'Autobianchi'
               ]
main_brands =[
    'Volkswagen', 'BMW', 'Audi', 'Opel', 'Ford', 'Mercedes-Benz', 'Renault', 'Toyota', 'Škoda', 'Peugeot', 'Citroën',
    'Honda', 'Suzuki', 'Mitsubishi', 'Jeep', 'Dacia' 'Kia', 'Volvo', 'Hyundai', 'Fiat', 'Seat', 'Nissan', 'Mazda',
]
global_mean_price = 63324

In [2135]:
counted_brands = df_car_sales["Marka_pojazdu"].value_counts()
# main_brands = counted_brands[counted_brands >= 1000].index.tolist()
# rest_brands = counted_brands[counted_brands < 1000].index.tolist()
main_brands

['Volkswagen',
 'BMW',
 'Audi',
 'Opel',
 'Ford',
 'Mercedes-Benz',
 'Renault',
 'Toyota',
 'Škoda',
 'Peugeot',
 'Citroën',
 'Honda',
 'Suzuki',
 'Mitsubishi',
 'Jeep',
 'DaciaKia',
 'Volvo',
 'Hyundai',
 'Fiat',
 'Seat',
 'Nissan',
 'Mazda']

In [2136]:
def create_one_hot_encoding_for_brands(df_car_sales):
    brands_encoding_df = pd.DataFrame()

    brands_encoding_df["Marka_pojazdu"] = df_car_sales["Marka_pojazdu"]
    # Kolumna pomocnicza do one-hot encoding
    brands_encoding_df["Main_brand"] = df_car_sales["Marka_pojazdu"].apply(
        lambda x: x if str(x) in main_brands else np.nan
    )

    # Kolumna z encodingiem dla pozostałych marek
    brands_encoding_df["Marka_pojazdu_encoded"] = brands_encoding_df['Marka_pojazdu'].apply(
        lambda x: str(less_common_brands_average_prices.get(str(x), global_mean_price)/global_mean_price) if str(x) not in main_brands else 1
    )
    # One-hot encoding dla głównych marek
    one_hot = pd.get_dummies(brands_encoding_df['Main_brand'], prefix='Marka', dummy_na=False, dtype="float")
    # Łączenie danych
    brands_encoding_df = brands_encoding_df.join(one_hot)
    brands_encoding_df.drop(['Marka_pojazdu', "Main_brand"], axis=1, inplace=True)
    return brands_encoding_df

In [2137]:
brands_encoding_df = create_one_hot_encoding_for_brands(df_car_sales)
# df_car_sales = df_car_sales.join(brands_encoding_df)

Dodanie kierownicy

In [2138]:
kraje_prawa_strona = ['Great Britain', 'Ireland', 'Japan', 'Australia', 'India', 'South Africa']

# Tworzenie mapowania
kierownica_mapping = {
    kraj: 1 if kraj in kraje_prawa_strona else 0
    for kraj in df_car_sales['Kraj_pochodzenia'].unique()
}

# Dodaj nową kolumnę
df_car_sales['Kierownica_strona'] = df_car_sales['Kraj_pochodzenia'].map(kierownica_mapping)

# Wyświetl wynik
df_car_sales

,Cena,Waluta,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,Kolor,...,SUV,city_cars,compact,convertible,coupe,minivan,sedan,small_cars,station_wagon,Kierownica_strona
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,1,Renault,Grand Espace,Gr 2.0T 16V Expression,2005.0,213000.0,170.0,1998.0,blue,...,False,False,False,False,False,True,False,False,False,0
2,25900.0,1,Renault,Megane,1.6 16V 110,2010.0,117089.0,110.0,1598.0,silver,...,False,False,False,False,False,False,False,False,True,0
3,35900.0,1,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,2015.0,115600.0,136.0,1598.0,white,...,False,False,False,False,False,True,False,False,False,0
4,5999.0,1,Ford,Focus,1.6 TDCi FX Silver / Silver X,2007.0,218000.0,90.0,1560.0,blue,...,False,False,True,False,False,False,False,False,False,0
5,44800.0,1,Toyota,Avensis,1.8,2013.0,NaN,NaN,1798.0,other,...,False,False,False,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,1,Opel,Astra,<NA>,2018.0,136931.0,150.0,1598.0,silver,...,False,False,True,False,False,False,False,False,False,0
135394,269855.0,1,Mercedes-Benz,Vito,124 CDI Tourer Lang,2021.0,8.0,237.0,1950.0,white,...,False,False,False,False,False,True,False,False,False,0
135395,21900.0,1,<NA>,Zafira,<NA>,NaN,179000.0,120.0,1700.0,black,...,False,False,False,False,False,True,False,False,False,0


# Encoding nieliczbowych klas

In [2139]:
df_car_sales.iloc[:, :20].info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Cena                           135397 non-null  float64       
 1   Waluta                         135397 non-null  int32         
 2   Marka_pojazdu                  132046 non-null  string        
 3   Model_pojazdu                  132088 non-null  string        
 4   Wersja_pojazdu                 87336 non-null   string        
 5   Rok_produkcji                  125572 non-null  float64       
 6   Przebieg_km                    131394 non-null  float64       
 7   Moc_KM                         131664 non-null  float64       
 8   Pojemnosc_cm3                  130711 non-null  float64       
 9   Kolor                          131934 non-null  category      
 10  Kraj_pochodzenia               74977 non-null   category      
 11  Pierw

In [2140]:
df_car_sales.iloc[:, 20:].info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 100 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Typ_silnika                      48194 non-null   string 
 1   On-board computer                132144 non-null  boolean
 2   Dual zone air conditioning       132144 non-null  boolean
 3   Roof rails                       132144 non-null  boolean
 4   GPS navigation                   132144 non-null  boolean
 5   Heated front seats               132144 non-null  boolean
 6   Multifunction steering wheel     132144 non-null  boolean
 7   ASR (traction control)           132144 non-null  boolean
 8   Bluetooth                        132144 non-null  boolean
 9   Velor upholstery                 132144 non-null  boolean
 10  Front parking sensors            132144 non-null  boolean
 11  Panoramic roof                   132144 non-null  boolean
 12  Cruise

In [2141]:
number_columns = []
for col in df_car_sales.columns:
    col_type = df_car_sales[col].dtype
    if pd.api.types.is_numeric_dtype(col_type):
        if pd.api.types.is_integer_dtype(col_type) or pd.api.types.is_float_dtype(col_type):
            number_columns.append(col)


In [2142]:
df_car_sales[number_columns].max()

Cena                         6.999000e+06
Waluta                       1.000000e+00
Rok_produkcji                2.021000e+03
Przebieg_km                  1.111111e+09
Moc_KM                       1.398000e+03
Pojemnosc_cm3                8.400000e+03
Pierwszy_wlasciciel          1.000000e+00
Rok_publikacji_oferty        2.021000e+03
Miesiac_publikacji_oferty    5.000000e+00
Rok_startu_produkcji         2.020000e+03
Rok_konca_produkcji          2.021000e+03
Kierownica_strona            1.000000e+00
dtype: float64

In [2143]:
# Zbierz kolumny do encodowania
columns_to_encode = []
for column in df_car_sales.columns:
    if df_car_sales[column].dtype not in ['float64', 'int', "bool"]:
        columns_to_encode.append(column)
columns_to_encode

['Marka_pojazdu',
 'Model_pojazdu',
 'Wersja_pojazdu',
 'Kolor',
 'Kraj_pochodzenia',
 'Data_publikacji_oferty',
 'Rok_pierwszej_rejestracji',
 'Miesiac_pierwszej_rejestracji',
 'Nazwa_generacji',
 'Typ_silnika',
 'On-board computer',
 'Dual zone air conditioning',
 'Roof rails',
 'GPS navigation',
 'Heated front seats',
 'Multifunction steering wheel',
 'ASR (traction control)',
 'Bluetooth',
 'Velor upholstery',
 'Front parking sensors',
 'Panoramic roof',
 'Cruise control',
 'ESP(stabilization of the track)',
 'Electric rear windows',
 'Electrically adjustable seats',
 'Drivers airbag',
 'Sunroof',
 'Speed limiter',
 'Electric front windows',
 'Air curtains',
 'AUX socket',
 'Power steering',
 'TV tuner',
 'Alloy wheels',
 'HUD(head-up display)',
 'DVD player',
 'CD',
 'Heated side mirrors',
 'Hook',
 'Fog lights',
 'Shift paddles',
 'Adjustable suspension',
 'Factory radio',
 'MP3',
 'Rain sensor',
 'Aftermarket radio',
 'Alarm',
 'Electrochromic side mirrors',
 'Manual air conditi

In [2144]:
df_car_sales.drop(columns=["Rok_publikacji_oferty"], inplace=True)

In [2145]:
# print(df_car_sales.loc[df_car_sales["Rok_pierwszej_rejestracji"].notnull()].shape)
# print(df_car_sales.loc[df_car_sales["Miesiac_pierwszej_rejestracji"].notnull()].shape)
mask = df_car_sales.loc[df_car_sales["Rok_pierwszej_rejestracji"].notnull() & df_car_sales["Rok_pierwszej_rejestracji"].notnull()].index
df = pd.DataFrame()
df.loc[:, "Czas_rejestracji"] = df_car_sales.loc[mask, "Rok_pierwszej_rejestracji"] +  df_car_sales.loc[mask, "Miesiac_pierwszej_rejestracji"]/13
df["Czas_rejestracji"] = df.loc[:, "Czas_rejestracji"].astype('float')
df.loc[:, "Cena"] = df_car_sales.loc[mask, "Cena"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54699 entries, 2 to 135397
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Czas_rejestracji  54699 non-null  float64
 1   Cena              54699 non-null  float64
dtypes: float64(2)
memory usage: 1.3 MB


In [2146]:
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# # Konwersja kolumny "Cena" na typ numeryczny, usuwając niepotrzebne znaki (np. spacje, złotówki)
#
#
#
# # Wybór tylko kolumn numerycznych do obliczenia korelacji
# numeric_columns = df.select_dtypes(include=[np.number]).columns
#
# # Obliczenie macierzy korelacji
# correlation_matrix = df[numeric_columns].corr()
#
# # Wyświetlenie macierzy korelacji dla kolumny "Cena"
# cena_correlation = correlation_matrix['Cena'].sort_values(ascending=False)
#
# # Wizualizacja macierzy korelacji
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
# plt.title('Macierz korelacji')
# plt.show()
#
# # Wyświetlenie korelacji z kolumną "Cena"
# print(cena_correlation)

In [2147]:
df_car_sales.drop(columns=["Kolor", "Miesiac_pierwszej_rejestracji", "Rok_pierwszej_rejestracji"], inplace=True)


In [2148]:
bool_cols = df_car_sales.select_dtypes(include='bool').columns
df_car_sales[bool_cols] = df_car_sales[bool_cols].astype(float)


Note:
- zastnanowić się czy wartości numerycznych nie zamienić na przedziały
- Uwzględnić nieproporcjonalność danych, rozważyć zastosowanie Oversamplingu


- Kiedy NIE usuwać wierszy?

    Gdy braki są systematyczne (np. wszystkie samochody marki "Opel" mają brakującą Emisja_CO2) – usuwanie wprowadzi bias.

    Gdy dane są rzadkie (np. tylko 100 wierszy), a usunięcie wierszy znacząco zmniejszy próbkę.

    Gdy można uzupełnić braki (np. średni Przebieg_km dla danej marki/roku).


In [2149]:
def remove_outliers(df, columns, lower_quantile=0.05, upper_quantile=0.95):
    df_clean = df.copy()
    for col in columns:
        lower_bound = df_clean[col].quantile(lower_quantile)
        upper_bound = df_clean[col].quantile(upper_quantile)
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

numeric_columns = ['Cena', 'Rok_produkcji', 'Przebieg_km', 'Moc_KM', 'Pojemnosc_cm3']

# df_no_outliers = remove_outliers(df_car_sales, numeric_columns, lower_quantile=0.05, upper_quantile=0.95)
# print("Wymiary po usunięciu outlierów:", df_no_outliers.shape)

Wymiary po usunięciu outlierów: (73108, 116)


In [2150]:
def fill_missing_with_median(df, columns):
    df_filled = df.copy()
    for col in columns:
        median_value = df_filled[col].median()
        df_filled[col] = df_filled[col].fillna(median_value)
    return df_filled

# df_filled = fill_missing_with_median(df_car_sales, numeric_columns)
# print("Liczba brakujących wartości po imputacji:")
# print(df_filled[numeric_columns].isnull().sum())

In [2151]:
# df_train_cleaned = df_car_sales.copy()
# df_test_cleaned = df_car_sales_test_data.copy()

# df_car_sales.to_csv("Cleaned_train.csv", index=True)
df_car_sales.to_csv("Cleaned_train.csv", index=True)

In [2152]:
['Kraj_pochodzenia',
 'Cena',
 'Rok_produkcji',
 'Wersja_pojazdu',
 'Data_publikacji_oferty',
 'Waluta',
 'Moc_KM',
 'Przebieg_km',
 'Marka_pojazdu',
 'Pierwszy_wlasciciel',
 'Miesiac_publikacji_oferty',
 'Model_pojazdu',
 'Pojemnosc_cm3']

['Kraj_pochodzenia',
 'Cena',
 'Rok_produkcji',
 'Wersja_pojazdu',
 'Data_publikacji_oferty',
 'Waluta',
 'Moc_KM',
 'Przebieg_km',
 'Marka_pojazdu',
 'Pierwszy_wlasciciel',
 'Miesiac_publikacji_oferty',
 'Model_pojazdu',
 'Pojemnosc_cm3']

In [2153]:
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Columns: 116 entries, Cena to Kierownica_strona
dtypes: category(1), datetime64[ns](1), float64(105), int32(1), int64(1), object(2), string(5)
memory usage: 123.5+ MB


In [2154]:
df_car_sales.columns.tolist()

['Cena',
 'Waluta',
 'Marka_pojazdu',
 'Model_pojazdu',
 'Wersja_pojazdu',
 'Rok_produkcji',
 'Przebieg_km',
 'Moc_KM',
 'Pojemnosc_cm3',
 'Kraj_pochodzenia',
 'Pierwszy_wlasciciel',
 'Data_publikacji_oferty',
 'Miesiac_publikacji_oferty',
 'Nazwa_generacji',
 'Rok_startu_produkcji',
 'Rok_konca_produkcji',
 'Typ_silnika',
 'On-board computer',
 'Dual zone air conditioning',
 'Roof rails',
 'GPS navigation',
 'Heated front seats',
 'Multifunction steering wheel',
 'ASR (traction control)',
 'Bluetooth',
 'Velor upholstery',
 'Front parking sensors',
 'Panoramic roof',
 'Cruise control',
 'ESP(stabilization of the track)',
 'Electric rear windows',
 'Electrically adjustable seats',
 'Drivers airbag',
 'Sunroof',
 'Speed limiter',
 'Electric front windows',
 'Air curtains',
 'AUX socket',
 'Power steering',
 'TV tuner',
 'Alloy wheels',
 'HUD(head-up display)',
 'DVD player',
 'CD',
 'Heated side mirrors',
 'Hook',
 'Fog lights',
 'Shift paddles',
 'Adjustable suspension',
 'Factory radi